**Prep Script:** assign capacities for all periods (from completed scenario) so operations can be run under, e.g., changing the level of trade allowed. <a id='top'></a>

In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [2]:
# path to completed scenarios
respath = '../outputs/'

# get list of completed scenarios (if multiple), and sort as needed. can also specify manually
scen_list = [i for i in os.listdir(respath) if '_tn' in i]

def myFunc(e):
    return len(e)
scen_list.sort(key=myFunc)

scen_list

['prm15_wcost_re1_c2p1_tn',
 'prm15_wcost_re2_c2p1_tn',
 'prm15_wcost_re3_c2p1_tn',
 'prm15_wcost_re4_c2p1_tn',
 'prm15_wcost_re5_c2p1_tn',
 'prm15_wcost_re6_c2p1_tn',
 'prm15_wcost_re7_c2p1_tn',
 'prm15_wcost_re8_c2p1_tn',
 'prm15_wcost_re9_c2p1_tn',
 'prm15_wcost_re10_c2p1_tn',
 'prm15_wcost_re11_c2p1_tn',
 'prm15_wcost_re12_c2p1_tn',
 'prm15_wcost_re13_c2p1_tn',
 'prm15_wcost_re14_c2p1_tn',
 'prm15_wcost_re15_c2p1_tn',
 'prm15_wcost_re16_c2p1_tn',
 'prm15_wcost_re17_c2p1_tn',
 'prm15_wcost_re18_c2p1_tn',
 'prm15_wcost_re19_c2p1_tn',
 'prm15_wcost_re20_c2p1_tn',
 'prm15_wcost_re1_c2p1_co2_cut90p_tn',
 'prm15_wcost_re2_c2p1_co2_cut90p_tn',
 'prm15_wcost_re3_c2p1_co2_cut90p_tn',
 'prm15_wcost_re4_c2p1_co2_cut90p_tn',
 'prm15_wcost_re5_c2p1_co2_cut90p_tn',
 'prm15_wcost_re6_c2p1_co2_cut90p_tn',
 'prm15_wcost_re7_c2p1_co2_cut90p_tn',
 'prm15_wcost_re8_c2p1_co2_cut90p_tn',
 'prm15_wcost_re9_c2p1_co2_cut90p_tn',
 'prm15_wcost_re10_c2p1_co2_cut90p_tn',
 'prm15_wcost_re11_c2p1_co2_cut90p_tn'

In [3]:
# path to 'project_specified_fixed_cost' input file, including file name
speccost = pd.read_csv('data/1_project_existing_fixed_cost_1.csv')

# path to 'project_new_potential' input file, including file name
newpot = pd.read_csv('data/1_project_new_potential_1.csv')

# path to 'project_portfolios' input file, including file name
portf = pd.read_csv('data/1_project_portfolios_1.csv')

# path to write new files
outpath = 'new_inputs/'

# set scenario id for new files (will increment if multiple scenarios passed)
id0 = 101
ids = np.arange(id0, id0+len(scen_list))

# for project_portfolio and project_new_potential, a scenario ID of "0" will be used to signify no future expansion.
# further, it will be the same for any scenario created using this method, so no need for multiple ID's.

In [4]:
i=0 # for testing, before creating loop.

***

### 1. Set new project potentials to zero

In [5]:
newpot['max_cumulative_new_build_mw'] = 0
newpot.to_csv(outpath+'0_project_new_potential_0.csv',index=False)
newpot.head()

project  period  min_cumulative_new_build_mw  \
0    P_Argentina_Biomass    2020                            0   
1   P_Argentina_Coal_Sub    2020                            0   
2   P_Argentina_Coal_Sup    2020                            0   
3  P_Argentina_Coal_IGCC    2020                            0   
4   P_Argentina_Gas_OCGT    2020                            0   

   max_cumulative_new_build_mw  min_cumulative_new_build_mwh  \
0                            0                           NaN   
1                            0                           NaN   
2                            0                           NaN   
3                            0                           NaN   
4                            0                           NaN   

   max_cumulative_new_build_mwh  
0                           NaN  
1                           NaN  
2                           NaN  
3                           NaN  
4                           NaN

***

### 2. Update project portfolio so that all projects are "gen_spec" (or "stor_spec")

In [6]:
portf['capacity_type'] = portf['capacity_type'].map({'gen_spec':'gen_spec','gen_new_lin':'gen_spec',
                                                     'gen_new_bin':'gen_spec','stor_new_lin':'stor_spec'})
portf.to_csv(outpath+'0_project_portfolios_0.csv',index=False)
portf.capacity_type.value_counts()

gen_spec     2963
stor_spec       5
Name: capacity_type, dtype: int64

***

### 3. Format newly built capacity file into existing capacity base input
### and
### 4. Update project specified fixed costs with new capacity costs

In [33]:
for i in range(len(scen_list)):
    newcap = pd.read_csv(respath+'{}/results/capacity_all.csv'.format(scen_list[i]))
    
    speccap = newcap[['project','period','capacity_mw','capacity_mwh']]\
                      .rename(columns={'capacity_mwh':'specified_capacity_mwh',
                                       'capacity_mw':'specified_capacity_mw'})

    speccap.to_csv(outpath + str(ids[i]) + '_project_existing_capacity_base.csv', index=False)
    
    cost_new = pd.read_csv(respath+'{}/results/costs_capacity_all_projects.csv'.format(scen_list[i]))
    
    cost_cap = cost_new.drop(columns=['technology','load_zone'])\
                   .merge(speccap.drop(columns='specified_capacity_mwh'), on=['project','period'])

    cost_cap['annual_fixed_cost_per_mw_year'] = (cost_cap['capacity_cost']/cost_cap['specified_capacity_mw'])*\
                                                (cost_cap['hours_in_full_period']/cost_cap['hours_in_subproblem_period'])

    cost_cap['annual_fixed_cost_per_mwh_year'] = 0
    cost_cap = cost_cap[['project','period','annual_fixed_cost_per_mw_year','annual_fixed_cost_per_mwh_year']]
    
    cost_cap = cost_cap.fillna(0)
    cost_cap.to_csv(outpath + '{}_project_existing_fixed_cost_{}.csv'.format(str(ids[i]),str(ids[i])), index=False)

cost_cap

project  period  annual_fixed_cost_per_mw_year  \
0      E_Argentina_Biomass_2015_rest    2020                  122700.000000   
1      E_Argentina_Biomass_2015_rest    2025                  122700.000000   
2      E_Argentina_Biomass_2015_rest    2030                  122700.000000   
3      E_Argentina_Biomass_2015_rest    2035                  122700.000000   
4      E_Argentina_Biomass_2015_rest    2040                  122700.000000   
...                              ...     ...                            ...   
20771          P_Hydro_Tocantins1310    2040                  146753.573298   
20772             P_Hydro_Amazon1082    2050                  146753.573298   
20773    U_Hydro_SAmerica_Colorado89    2045                       0.000000   
20774          P_Hydro_Tocantins1297    2030                       0.000000   
20775                P_Hydro_Negro53    2035                  146753.573298   

       annual_fixed_cost_per_mwh_year  
0                                   0  
1                                   0  
2                                   0  
3                                   0  
4                                   0  
...                               ...  
20771                               0  
20772                               0  
20773                               0  
20774                               0  
20775                               0  

[20776 rows x 4 columns]

***

### Next Steps:
Update scenarios file with new scenario ID's<br>
Rerun GridPath

[Return to Top](#top)